In [10]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [1]:
import pandas as pd

# Load the final merged hourly dataset
merged_data = pd.read_parquet('../datasets/final_merged_dataset_hourly.parquet')

# Confirm it's loaded properly
print(merged_data.head())
print(f"Total data points: {len(merged_data)}")

             datetime      Open      High       Low     Close    Volume  \
0 2021-01-01 00:00:00  0.250730  0.251106  0.250628  0.250493  0.076344   
1 2021-01-01 01:00:00  0.251433  0.251695  0.251353  0.251520  0.015258   
2 2021-01-01 02:00:00  0.256154  0.256011  0.255263  0.255877  0.034476   
3 2021-01-01 03:00:00  0.252147  0.252928  0.252351  0.253123  0.010795   
4 2021-01-01 04:00:00  0.254932  0.255284  0.254180  0.254066  0.023016   

   sentiment_score  bert_sentiment  prev_close  prev_vader_sentiment  \
0              0.0             0.0    0.250493                   0.0   
1              0.0             0.0    0.250493                   0.0   
2              0.0             0.0    0.251520                   0.0   
3              0.0             0.0    0.255877                   0.0   
4              0.0             0.0    0.253123                   0.0   

   prev_bert_sentiment  volatility_7d  volatility_14d  volatility_30d  \
0                  0.0       0.050683      

In [8]:
from neuralforecast.models import PatchTST
from neuralforecast import NeuralForecast

In [3]:
# Convert for PatchTST target df
df = merged_data[['datetime', 'Close']].copy()
df.rename(columns={'datetime': 'ds', 'Close': 'y'}, inplace=True)
df['unique_id'] = 'BTC'  # Single series
df = df[['unique_id', 'ds', 'y']]

In [ ]:
merged_data[['datetime']+['closing_7d_avg']]åç

,datetime,closing_7d_avg
0,2021-01-01 00:00:00,0.320975
1,2021-01-01 01:00:00,0.320975
2,2021-01-01 02:00:00,0.320975
3,2021-01-01 03:00:00,0.320975
4,2021-01-01 04:00:00,0.320975
...,...,...
18260,2023-01-31 20:00:00,0.140301
18261,2023-01-31 21:00:00,0.140323
18262,2023-01-31 22:00:00,0.140352
18263,2023-01-31 23:00:00,0.140378


In [12]:
exogenous_cols = [
    'sentiment_score', 'bert_sentiment',
    'prev_vader_sentiment', 'prev_bert_sentiment',
    'volatility_7d', 'volatility_14d', 'volatility_30d',
    'closing_7d_avg'
]

X_df = merged_data[['datetime'] + exogenous_cols].copy()
X_df.rename(columns={'datetime': 'ds'}, inplace=True)
X_df['unique_id'] = 'BTC'
X_df = X_df[['unique_id', 'ds'] + exogenous_cols]

In [13]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.losses.pytorch import MSE

patchtst = PatchTST(
    h=24,
    input_size=168,
    patch_len=16,
    stride=8,
    encoder_layers=3,           # 🔁 Replaces n_blocks
    n_heads=4,
    hidden_size=128,            # 🔁 Replaces d_model
    linear_hidden_size=256,     # Optional, helps model capacity
    dropout=0.1,
    fc_dropout=0.1,
    head_dropout=0.0,
    learning_rate=1e-3,
    scaler_type='robust',
    loss=MSE(),
    random_seed=42
)

nf = NeuralForecast(models=[patchtst], freq='H')
nf.fit(df=df)

Seed set to 42


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

NotImplementedError: The operator 'aten::nanmedian.dim_values' is not currently implemented for the MPS device. If you want this op to be considered for addition please comment on https://github.com/pytorch/pytorch/issues/141287 and mention use-case, that resulted in missing op as well as commit hash 134179474539648ba7dee1317959529fbd0e7f89. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.